In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import pandas as pd
import random

### MongoDB

In [2]:
uri = 'mongodb://localhost/'
db = MongoClient(uri)['vendas']

### estoque

In [3]:
xls = pd.read_excel('estoque.xlsx')

In [4]:
estoques = []
for i, e in enumerate(xls.to_dict('records')):
  estoques.append({
    'id': i + 1,
    'marca': e['marca'],
    'categoria': e['categoria'],
    'descricao': e['descricao'],
    'quantidade': e['quantidade'],
    'preco_compra': round(e['preco_compra'] * (random.random() * 0.5 + 0.2), 2),
    'preco_venda': e['preco_compra'],
  })
estoques

[{'id': 1,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1200W',
  'quantidade': 1000,
  'preco_compra': 39.68,
  'preco_venda': 149.73},
 {'id': 2,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1200W',
  'quantidade': 1000,
  'preco_compra': 57.94,
  'preco_venda': 169.9},
 {'id': 3,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1400W',
  'quantidade': 1000,
  'preco_compra': 138.18,
  'preco_venda': 460.35},
 {'id': 4,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1400W (Preto)',
  'quantidade': 1000,
  'preco_compra': 326.67,
  'preco_venda': 469.26},
 {'id': 5,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1500W',
  'quantidade': 1000,
  'preco_compra': 112.72,
  'preco_venda': 392.14},
 {'id': 6,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1500W',
 

In [5]:
db['venda_estoque'].delete_many({})
db['venda_estoque'].insert_many(estoques)

InsertManyResult([ObjectId('66522e4adeb1d60916062296'), ObjectId('66522e4adeb1d60916062297'), ObjectId('66522e4adeb1d60916062298'), ObjectId('66522e4adeb1d60916062299'), ObjectId('66522e4adeb1d6091606229a'), ObjectId('66522e4adeb1d6091606229b'), ObjectId('66522e4adeb1d6091606229c'), ObjectId('66522e4adeb1d6091606229d'), ObjectId('66522e4adeb1d6091606229e'), ObjectId('66522e4adeb1d6091606229f'), ObjectId('66522e4adeb1d609160622a0'), ObjectId('66522e4adeb1d609160622a1'), ObjectId('66522e4adeb1d609160622a2'), ObjectId('66522e4adeb1d609160622a3'), ObjectId('66522e4adeb1d609160622a4'), ObjectId('66522e4adeb1d609160622a5'), ObjectId('66522e4adeb1d609160622a6'), ObjectId('66522e4adeb1d609160622a7'), ObjectId('66522e4adeb1d609160622a8'), ObjectId('66522e4adeb1d609160622a9'), ObjectId('66522e4adeb1d609160622aa'), ObjectId('66522e4adeb1d609160622ab'), ObjectId('66522e4adeb1d609160622ac'), ObjectId('66522e4adeb1d609160622ad'), ObjectId('66522e4adeb1d609160622ae'), ObjectId('66522e4adeb1d609160622

# Pedidos

### base original

In [6]:
df_ori = pd.read_csv('Sales_Product_Combined.csv')
df_ori

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,19-04-2019,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,07-04-2019,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,30-04-2019,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones
...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,17-09-2019,8:56 PM,"840 Highland St, Los Angeles, CA 90001",Los Angeles,Batteries
185946,259354,iPhone,1,700,01-09-2019,4:00 PM,"216 Dogwood St, San Francisco, CA 94016",San Francisco,iPhone
185947,259355,iPhone,1,700,23-09-2019,7:39 AM,"220 12th St, San Francisco, CA 94016",San Francisco,iPhone
185948,259356,34in Ultrawide Monitor,1,379.99,19-09-2019,5:30 PM,"511 Forest St, San Francisco, CA 94016",San Francisco,Monitor


In [7]:
# quantidade por cidade
df_ori.groupby('City')['Quantity Ordered'].count().sort_values()

City
Austin            9905
Portland         12465
Seattle          14732
Dallas           14820
Atlanta          14881
Boston           19934
New York City    24876
Los Angeles      29605
San Francisco    44732
Name: Quantity Ordered, dtype: int64

In [8]:
df_ori.groupby('Product')['Quantity Ordered'].count().sort_values()

Product
LG Dryer                        646
LG Washing Machine              666
Vareebadd Phone                2065
20in Monitor                   4101
ThinkPad Laptop                4128
Macbook Pro Laptop             4724
Flatscreen TV                  4800
Google Phone                   5525
34in Ultrawide Monitor         6181
27in 4K Gaming Monitor         6230
iPhone                         6842
27in FHD Monitor               7507
Bose SoundSport Headphones    13325
Apple Airpods Headphones      15549
Wired Headphones              18882
AA Batteries (4-pack)         20577
AAA Batteries (4-pack)        20641
Lightning Charging Cable      21658
USB-C Charging Cable          21903
Name: Quantity Ordered, dtype: int64

### quantidades: Austin=9905, Boston=19934, San Francisco=44732

In [9]:
# renomeando campos
df_aux = df_ori.rename(columns={
  'Order ID': 'id', 
  'Product' : 'produto', 
  'Quantity Ordered': 'quantidade', 
  'Price': 'preco', 
  'Order Date': 'data',
  'Time': 'hora',
  'Product Type': 'categoria', 
  'Purchase Address': 'endereco', 
  'City': 'cidade',
})

# criando campo horário
df_aux['horario'] = pd.to_datetime(df_aux['data'] + ' ' + df_aux['hora'], format='%d-%m-%Y %I:%M %p')

# diminuindo quantidade de registros
df_aux['cidade'] = df_aux['cidade'].str.strip()
# df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "San Francisco"')[
df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "Austin"')[
  ['horario', 'produto', 'quantidade', 'preco']
]

### ajustando os valores

In [10]:
df = df_aux.sort_values('horario')

# campo auxiliar para aleatórios
df['rnd'] = [random.randrange(100) for _ in df.index]

# transferir 25% das vendas dos meses 3, 4 e 5 para os meses 9, 10 e 11
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-03-2019', format='%d-%m-%Y')) & 
  (df['horario'] < pd.to_datetime('01-06-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 25), 
  df['horario'] + pd.DateOffset(months=6))

# transferir 20% das vendas do mês 12 para o mês 1
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-12-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 20), 
  df['horario'] + pd.DateOffset(months=-11))

# distribuir aleatóriamente os registros em 5 anos
df['rnd'] = [random.randrange(100) for _ in df.index]
df['horario'] = df['horario'].mask(
  (df.rnd < 25), 
  df['horario'] + pd.DateOffset(years=2))
df['horario'] = df['horario'].mask(
  (df.rnd >= 25) & 
  (df.rnd < 50),
  df['horario'] + pd.DateOffset(years=3))
df['horario'] = df['horario'].mask(
  (df.rnd >= 50) & 
  (df.rnd < 75),
  df['horario'] + pd.DateOffset(years=4))
df['horario'] = df['horario'].mask(
  (df.rnd >= 75),
  df['horario'] + pd.DateOffset(years=5))

# campo auxiliar de data
df['data'] = pd.to_datetime(df['horario']).dt.date

# distribuir os itens do pedido
df['nota'] = df['horario'].dt.strftime('%Y%m%d-%M').str.slice(stop=10)

# permitir cálculo com o preço
df['preco'] = df['preco'].str.replace(',', '').astype(float)
df['preco'] = df.apply(lambda row: row['preco'] * 10 if row['preco'] < 80 else row['preco'], axis=1)
df['preco'] = df['preco'].round(2)

# calcular aleatóriamente quantidade, mas valores menores tem peso maior
pop = [i + 1 for i in range(40)]
wei = [i * i + 1 for i in range(40, 0, -1)]
df['quantidade'] = random.choices(pop, wei, k=len(df.index))
df['quantidade'] = (df['quantidade'] / (df['quantidade'] * df['preco'] / 10000 + 1).astype(int)).astype(int)
df['total'] = df['quantidade'] * df['preco']
df['total'] = df['total'].round(2)

# renomear produtos
prod = {
  'LG Dryer': '',
  'LG Washing Machine': '',
  'Vareebadd Phone': '',
  '20in Monitor': '',
  'ThinkPad Laptop': '',
  'Macbook Pro Laptop': '',
  'Flatscreen TV': '',
  'Google Phone': '',
  '34in Ultrawide Monitor': 'P29',
  '27in 4K Gaming Monitor': 'P28',
  'iPhone': 'P27',
  '27in FHD Monitor': 'P26',
  'Bose SoundSport Headphones': 'P25',
  'Apple Airpods Headphones': 'P24',
  'Wired Headphones': 'P23',
  'Lightning Charging Cable': 'P22',
  'USB-C Charging Cable': 'P21',
  'AA Batteries (4-pack)': 'P20',
  'AAA Batteries (4-pack)': 'P19',
}

df['produto'] = df.apply(
  lambda row: 
    prod[row['produto']] if row['produto'] in [
      # 'LG Dryer', 
      # 'LG Washing Machine', 
      # 'Vareebadd Phone', 
      # '20in Monitor', 
      # 'ThinkPad Laptop', 
      # 'Macbook Pro Laptop', 
      # 'Flatscreen TV', 
      # 'Google Phone', 

      '34in Ultrawide Monitor',
      '27in 4K Gaming Monitor',
      'iPhone',

      '27in FHD Monitor',
      'Bose SoundSport Headphones',
      'Apple Airpods Headphones',
      'Wired Headphones',
      'Lightning Charging Cable',
      'USB-C Charging Cable',
      'AA Batteries (4-pack)',
      'AAA Batteries (4-pack)'
    ] else f'P{random.randrange(1, 19):02}', axis=1)

df = df[['nota', 'data', 'horario', 'produto', 'quantidade']].sort_values('horario')
df

,nota,data,horario,produto,quantidade
76286,20210101-1,2021-01-01,2021-01-01 13:10:00,P23,26
73698,20210101-3,2021-01-01,2021-01-01 13:30:00,P19,3
69596,20210101-3,2021-01-01,2021-01-01 15:39:00,P25,8
68358,20210101-0,2021-01-01,2021-01-01 23:08:00,P21,3
36190,20210102-4,2021-01-02,2021-01-02 03:49:00,P20,16
...,...,...,...,...,...
44965,20241231-4,2024-12-31,2024-12-31 15:40:00,P24,1
43567,20241231-0,2024-12-31,2024-12-31 17:02:00,P26,5
48868,20241231-1,2024-12-31,2024-12-31 17:10:00,P22,29
33147,20241231-2,2024-12-31,2024-12-31 19:29:00,P22,34


In [11]:
# df.head()
# df.info()
# df[:20]
# df[-20:]
df.shape
# df_aux.shape
# df_aux[:20]
# df_aux[-20:]

(9903, 5)

In [12]:
# quantidade por mês
j = df[['horario', 'quantidade']]
j.index = df.horario
j.resample(rule='ME', on='horario')['quantidade'].sum()

horario
2021-01-31    1987
2021-02-28    1535
2021-03-31    1581
2021-04-30    1606
2021-05-31    1773
2021-06-30    1810
2021-07-31    1740
2021-08-31    1752
2021-09-30    2316
2021-10-31    3645
2021-11-30    2704
2021-12-31    2858
2022-01-31    1987
2022-02-28    1379
2022-03-31    1644
2022-04-30    1739
2022-05-31    1402
2022-06-30    1593
2022-07-31    1872
2022-08-31    1669
2022-09-30    2000
2022-10-31    3748
2022-11-30    2731
2022-12-31    2739
2023-01-31    2058
2023-02-28    1742
2023-03-31    1593
2023-04-30    1765
2023-05-31    1478
2023-06-30    1875
2023-07-31    2031
2023-08-31    1472
2023-09-30    2495
2023-10-31    3444
2023-11-30    2698
2023-12-31    3118
2024-01-31    1892
2024-02-29    1770
2024-03-31    1811
2024-04-30    1934
2024-05-31    1507
2024-06-30    1977
2024-07-31    2033
2024-08-31    1580
2024-09-30    2287
2024-10-31    3299
2024-11-30    2457
2024-12-31    2561
Freq: ME, Name: quantidade, dtype: int64

# Preparar e Inserir Pedidos

In [13]:
q = f'horario < "{datetime.now()}"'
df_ins = df.query(q).sort_values(['data', 'nota', 'produto'])

In [14]:
df_ins.head()
# df_ins[:30]
# df_ins[-30:]

,nota,data,horario,produto,quantidade
68358,20210101-0,2021-01-01,2021-01-01 23:08:00,P21,3
76286,20210101-1,2021-01-01,2021-01-01 13:10:00,P23,26
73698,20210101-3,2021-01-01,2021-01-01 13:30:00,P19,3
69596,20210101-3,2021-01-01,2021-01-01 15:39:00,P25,8
68579,20210102-0,2021-01-02,2021-01-02 23:01:00,P23,31


In [15]:
df_ins.shape

(8248, 5)

In [16]:
representantes = list(db['venda_representante'].find())

estoques = {}
for e in db['venda_estoque'].find():
  estoques[f"P{e['id']:02}"] = e

In [17]:
estoques

{'P01': {'_id': ObjectId('66522e4adeb1d60916062296'),
  'id': 1,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1200W',
  'quantidade': 1000,
  'preco_compra': 39.68,
  'preco_venda': 149.73},
 'P02': {'_id': ObjectId('66522e4adeb1d60916062297'),
  'id': 2,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1200W',
  'quantidade': 1000,
  'preco_compra': 57.94,
  'preco_venda': 169.9},
 'P03': {'_id': ObjectId('66522e4adeb1d60916062298'),
  'id': 3,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1400W',
  'quantidade': 1000,
  'preco_compra': 138.18,
  'preco_venda': 460.35},
 'P04': {'_id': ObjectId('66522e4adeb1d60916062299'),
  'id': 4,
  'marca': 'KazeKami',
  'categoria': 'Secadores',
  'descricao': 'Secador de Parede 1400W (Preto)',
  'quantidade': 1000,
  'preco_compra': 326.67,
  'preco_venda': 469.26},
 'P05': {'_id': ObjectId('66522e4adeb1d6091606229a'),
  'id': 5,
  

In [18]:
def gen_pedido(i, itens_pedido, total, representantes, horario):
  representante = random.choice(representantes)
  cliente = random.choice(representante['clientes'])

  etapas = [{
    'id': 1,
    'etapa': 'criado',
    'horario': horario - timedelta(seconds = random.randrange(9999, 99999)),
    'user_id': representante['id']
  }]
  
  etapa_id = 2
  if representante['id'] != 2:
    etapas.append({
      'id': etapa_id,
      'etapa': 'enviado',
      'horario': horario - timedelta(seconds = random.randrange(99, 9999)),
      'user_id': representante['id']
    })
    etapa_id += 1

  etapas.append({
    'id': etapa_id,
    'etapa': 'confirmado',
    'horario': horario,
    'user_id': 2
  })

  pedido = {
    'id': i,
    'etapa': 'confirmado',
    'etapas_pedido': etapas,
    'representante_id': representante['id'],
    'cliente': cliente,
    'horario': horario,
    'itens_pedido': itens_pedido,
    'total': round(total, 2)
  }
  return pedido


In [19]:
pedidos = []
itens_pedido = []
i = 0
nota = df_ins.iloc[0, 0]
horario = df_ins.iloc[0, 2]
prod_ant = ''
total = 0
for e in df_ins.itertuples():
  if nota != e.nota:
    i += 1
    pedido = gen_pedido(i, itens_pedido, total, representantes, horario)
    pedidos.append(pedido)
    itens_pedido = []
    prod_ant = ''
    total = 0
    nota = e.nota
    horario  = e.horario

  estoque = estoques[e.produto]
  total_prod = round(e.quantidade * estoque['preco_venda'], 2)
  total += total_prod
  if e.produto == prod_ant:
    itens_pedido[-1]['quantidade'] += e.quantidade
  else:
    item = {
      'id': estoque['id'],
      'produto': {
        'marca': estoque['marca'],
        'categoria': estoque['categoria'],
        'descricao': estoque['descricao']
        },
      'preco_compra': estoque['preco_compra'],
      'preco_venda': estoque['preco_venda'],
      'quantidade': e.quantidade,
      'total': total_prod
    }
    itens_pedido.append(item)
    prod_ant = e.produto

if nota != df_ins.iloc[0, 0]:
  pedido = gen_pedido(i + 1, itens_pedido, total, representantes, horario)
  pedidos.append(pedido)

In [20]:
# from pprint import pprint
# pprint(pedidos)

In [21]:
db['venda_pedido'].delete_many({})
db['venda_pedido'].insert_many(pedidos)

# atualizando controle do sequencial de pedidos
db['__schema__'].update_one({'name': 'venda_pedido'}, {
  '$set': {
    'auto.seq': len(pedidos)
  }}, upsert=False)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# mantendo a diferença com 1000 no estoque
for k in estoques.keys():
  estoques[k]['quant_venda'] = 0

for p in pedidos:
  for i in p['itens_pedido']:
    estoques[f'P{i["id"]:02}']['quant_venda'] += i['quantidade']

for k in estoques.keys():
  q = estoques[k]['quant_venda']

  db['venda_estoque'].update_one({'id': estoques[k]['id']}, {
    '$set': {
      'quantidade': (q // 1000 + 1) * 1000 - q
    }}, upsert=False)

# Preparar e Inserir Rastreios

In [23]:
from faker import Faker
fake = Faker('pt_BR')

trans = [
  'Cardoso S.A.', 
  'Rocha - ME', 
  'Novaes S.A.', 
  'Peixoto Ferreira Ltda.', 
  'Ramos da Rosa S.A.', 
  'Ribeiro Freitas - EI']
func1 = [
  'o Ian Viana',
  'a Letícia Fogaça',
  'o Vitor Hugo Farias',
  'a Bárbara das Neves',
  'o Carlos Eduardo Azevedo',
  'o Davi Luiz Castro',
  'a Gabriela Caldeira',
  'o Levi Gomes',]
func2 = [
  'Guilherme Azevedo',
  'Leonardo Nunes',
  'João Vitor Rezende',
  'Pedro Souza',
  'Thiago Teixeira',
  'Vitor Lima',
  'Otávio Ramos',
  'Cauã Martins',
  'Raul Nunes',
  'Luiz Henrique Souza']

In [24]:
def das_8_as_5(h, s):
  h += timedelta(seconds = s)
  if h.hour < 8: h += timedelta(hours = 8 - h.hour)
  if h.hour > 17: h += timedelta(hours = 32 - h.hour)
  return h

rastreios = []
for e in db['venda_pedido'].find():
  r = db['venda_representante'].find_one({"id": e['representante_id']})
  h1 = das_8_as_5(e['horario'], 0)
  h2 = das_8_as_5(h1, random.randrange(500, 9999))
  h3 = das_8_as_5(h2, random.randrange(1999, 99999))
  h4 = das_8_as_5(h3, random.randrange(1999, 99999))
  h5 = das_8_as_5(h4, random.randrange(500, 1999))
  h6 = das_8_as_5(h5, random.randrange(500, 1999))
  etapas_rastreio = [{
    "id": 1,
    "titulo": "Recebemos o seu pedido",
    "descricao": f"Pedido gerado por {r['nome']}.",
    "horario": e['horario'],
    "icon": "fa-receipt"
  }, {
    "id": 2,
    "titulo": "Separando e embalando os produtos",
    "descricao": f"Olá, {random.choice(func1)} está quase terminando de embalar.",
    "horario": h2,
    "icon": "fa-box-open"
  }, {
    "id": 3,
    "titulo": "Enviando para a transportadora",
    "descricao": f"O pedido está sendo encaminhado à transportadora \"{random.choice(trans)}\" que irá realizar a entrega.",
    "horario": h3,
    "icon": "fa-truck-ramp-box"
  }, {
    "id": 4,
    "titulo": "Saindo para entrega",
    "descricao": f"Seu pedido já saiu e será entregue por {random.choice(func2)}.",
    "horario": h4,
    "icon": "fa-truck-arrow-right"
  }, {
    "id": 5,
    "titulo": "Chegando no endereço de destino",
    "descricao": f"Estou próximo ao destino ({e['cliente']['endereco']['logradouro']}) e pedimos a presença do responsável para receber o pedido.",
    "horario": h5,
    "icon": "fa-truck-fast"
  }, {
    "id": 6,
    "titulo": "Pedido entregue!",
    "descricao": f"O pedido foi entregue para {e['cliente']['contato']}",
    "horario": h6,
    "icon": "fa-list-check"
  }]

  rastreios.append({
    "id": e['id'],
    "pedido_id": e['id'],
    "previsao": e['horario'] + timedelta(days = random.randrange(2, 5)),
    "etapa_atual": etapas_rastreio[5]["titulo"],
    "etapas_rastreio": etapas_rastreio,
  })

rastreios

[{'id': 1,
  'pedido_id': 1,
  'previsao': datetime.datetime(2021, 1, 4, 23, 8),
  'etapa_atual': 'Pedido entregue!',
  'etapas_rastreio': [{'id': 1,
    'titulo': 'Recebemos o seu pedido',
    'descricao': 'Pedido gerado por Kamilly Martins.',
    'horario': datetime.datetime(2021, 1, 1, 23, 8),
    'icon': 'fa-receipt'},
   {'id': 2,
    'titulo': 'Separando e embalando os produtos',
    'descricao': 'Olá, a Gabriela Caldeira está quase terminando de embalar.',
    'horario': datetime.datetime(2021, 1, 2, 8, 37, 34),
    'icon': 'fa-box-open'},
   {'id': 3,
    'titulo': 'Enviando para a transportadora',
    'descricao': 'O pedido está sendo encaminhado à transportadora "Cardoso S.A." que irá realizar a entrega.',
    'horario': datetime.datetime(2021, 1, 3, 8, 8, 55),
    'icon': 'fa-truck-ramp-box'},
   {'id': 4,
    'titulo': 'Saindo para entrega',
    'descricao': 'Seu pedido já saiu e será entregue por Cauã Martins.',
    'horario': datetime.datetime(2021, 1, 4, 8, 29, 54),
    

In [25]:
db['venda_rastreio'].delete_many({})
db['venda_rastreio'].insert_many(rastreios)

# atualizando controle do sequencial de rastreios
db['__schema__'].update_one({'name': 'venda_rastreio'}, {
  '$set': {
    'auto.seq': len(rastreios)
  }}, upsert=False)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)